<a href="https://colab.research.google.com/github/filmerxyz/JSTP-22_SkinDiseaseClassificationUsingMachineLearning/blob/master/Transfer_DenseNet121.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DenseNet121**

# Check GPU

In [1]:
!nvidia-smi

Tue May 26 08:06:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Install Split Folders

In [2]:
!pip install split-folders

# Import Libraries

In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import keras
from keras.models import Model, load_model
from keras.layers import Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import split_folders

import matplotlib.pyplot as plt

from pj61403_utils_functions import mkdir_dir, plot_confusion_matrix, normalize, deprocess_image, my_decode_predictions, guided_backprop, grad_cam, compute_saliency

# Data

## Clone Data

In [6]:
!git clone https://github.com/EvilPickle-PCSHSPT/PJ61403_DATA

Cloning into 'PJ61403_DATA'...
remote: Enumerating objects: 278, done.
remote: Counting objects: 100% (278/278), done.
remote: Compressing objects: 100% (250/250), done.
remote: Total 1444 (delta 60), reused 236 (delta 28), pack-reused 1166
Receiving objects: 100% (1444/1444), 149.83 MiB | 37.12 MiB/s, done.
Resolving deltas: 100% (401/401), done.


## Hyperparameters

In [0]:
IMG_WIDTH = 224
IMG_HEIGHT = 224

EPOCHS = 100

BATCH_SIZE = 32

LR = 1e-3

In [8]:
mkdir_dir('/content/data_folder')

Directory  /content/data_folder  Created 


### Split Data

In [9]:
split_folders.ratio('/content/PJ61403_DATA/classes_data', 
                    output='/content/data', 
                    seed=1337, 
                    ratio=(0.63, 0.27, 0.1)) #train=0.8, val=0.1, test=0.1

Copying files: 246 files [00:00, 1832.56 files/s]


In [0]:
train_dir = '/content/data/train'
val_dir = '/content/data/val'
test_dir = '/content/data/test'

### Data Augmentation

In [0]:
train_datagen = ImageDataGenerator(rescale=1. / 255, 
                                shear_range=0.2, 
                                zoom_range=0.2, 
                                horizontal_flip=True,
                                fill_mode='reflect')

val_datagen = ImageDataGenerator(rescale=1. / 255)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [14]:
train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    class_mode ='categorical')

val_generator = val_datagen.flow_from_directory(directory=val_dir,
                                                target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                                class_mode='categorical')

test_generator = test_datagen.flow_from_directory(directory=test_dir,
                                                  target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                  batch_size=1,
                                                  shuffle=False,
                                                  class_mode=None)

Found 152 images belonging to 4 classes.
Found 64 images belonging to 4 classes.
Found 30 images belonging to 4 classes.


## Base Model

In [15]:
base_model = load_model('/content/acc_densenet121_model.h5', compile=False)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.




In [0]:
# base_model.summary()

In [21]:
len(base_model.layers)

436

## New Model

In [0]:
for i in range(436):
    base_model.layers[i].trainable = False

fc = base_model.layers[434].output
fc = Dense(32, activation='relu')(fc)
fc = Dense(16, activation='relu')(fc)
prediction = Dense(4, activation='softmax')(fc)

new_model = Model(inputs=base_model.input,outputs=prediction)

new_model.compile(optimizer=Adam(LR), 
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [54]:
new_model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_9 (ZeroPadding2D (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_9[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
___________________________________________________________________________________________

## Callbacks

In [0]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

model_checkpoint = ModelCheckpoint(filepath='/content/tl_densenet121_model.h5', save_best_only=True, monitor='val_loss', mode='min')

reduce_lr_rate = ReduceLROnPlateau(monitor='val_loss', factor=0.3, min_lr=1e-5 , patience=10, verbose=1, min_delta=1e-4, mode='min')

## Training

In [56]:
history = new_model.fit(train_generator,
                        validation_data=val_generator,
                        epochs=EPOCHS,
                        callbacks=[early_stop, model_checkpoint, reduce_lr_rate])

Epoch 1/100
5/5 [==============================] - 13s 3s/step - loss: 1.6198 - accuracy: 0.2500 - val_loss: 1.4696 - val_accuracy: 0.2969
Epoch 2/100
5/5 [==============================] - 2s 379ms/step - loss: 1.4809 - accuracy: 0.3487 - val_loss: 1.3823 - val_accuracy: 0.3125
Epoch 3/100
5/5 [==============================] - 4s 877ms/step - loss: 1.4365 - accuracy: 0.3158 - val_loss: 1.3967 - val_accuracy: 0.3281
Epoch 4/100
5/5 [==============================] - 4s 867ms/step - loss: 1.4214 - accuracy: 0.3947 - val_loss: 1.4138 - val_accuracy: 0.3281
Epoch 5/100
5/5 [==============================] - 4s 858ms/step - loss: 1.3751 - accuracy: 0.4145 - val_loss: 1.4120 - val_accuracy: 0.3125
Epoch 6/100
5/5 [==============================] - 4s 870ms/step - loss: 1.3709 - accuracy: 0.4013 - val_loss: 1.4011 - val_accuracy: 0.2969
Epoch 7/100
5/5 [==============================] - 4s 878ms/step - loss: 1.3338 - accuracy: 0.4539 - val_loss: 1.3517 - val_accuracy: 0.2812
Epoch 8/100
5/5

KeyboardInterrupt: ignored

# Graph

## Accuraccy

In [60]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.title('accuracy')
plt.show()

NameError: ignored

## Loss

In [61]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.title('accuracy')
plt.show()

NameError: ignored

# Evaluate

In [0]:
test_model = load_model('/content/tl_densenet121_model.h5')